# Capstone project - HealthCare Fraud Detection - EDA and Clustering

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from cp_clean_helper import show_values
plt.style.use('ggplot')
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 100)

### # Loading cleaned training and testing datasets and fraud labels:-

In [2]:
train_df = pd.read_csv("clean_train.csv",index_col=0)
test_df = pd.read_csv("clean_test.csv",index_col=0)
fraud_labels = pd.read_csv("Data/Train-1542865627584.csv")

### # Looking at dataset shapes:-

In [3]:
print('Shape of fraud labels data :',fraud_labels.shape)
print('Shape of train data :',train_df.shape)
print('Shape of test data :',test_df.shape)

Shape of fraud labels data : (5410, 2)
Shape of train data : (558211, 59)
Shape of test data : (135392, 58)


### # Looking at first few rows for each:

In [4]:
train_df.head(5)

BeneID  Gender  Race  RenalDiseaseIndicator  State  County  \
0  BENE11001       0     0                      0     39     230   
1  BENE12511       0     0                      0     39     510   
2  BENE14577       1     0                      1     33      70   
3  BENE15074       1     0                      0     39     470   
4  BENE16973       0     0                      0     39     310   

   NoOfMonths_PartACov  NoOfMonths_PartBCov  ChronicCond_Alzheimer  \
0                   12                   12                      0   
1                   12                   12                      1   
2                   12                   12                      1   
3                   12                   12                      0   
4                   12                   12                      0   

   ChronicCond_Heartfailure  ChronicCond_KidneyDisease  ChronicCond_Cancer  \
0                         1                          0                   1   
1                         1                          1                   1   
2                         0                          0                   1   
3                         1                          0                   1   
4                         0                          0                   1   

   ChronicCond_ObstrPulmonary  ChronicCond_Depression  ChronicCond_Diabetes  \
0                           1                       0                     0   
1                           1                       1                     0   
2                           1                       1                     1   
3                           1                       1                     0   
4                           1                       1                     1   

   ChronicCond_IschemicHeart  ChronicCond_Osteoporasis  \
0                          0                         1   
1                          0                         0   
2                          0                         1   
3                          0                         1   
4                          0                         1   

   ChronicCond_rheumatoidarthritis  ChronicCond_stroke  OPAnnualDeductibleAmt  \
0                                0                   0                     70   
1                                1                   1                     90   
2                                1                   0                    440   
3                                1                   1                    210   
4                                1                   1                    200   

   Deceased  Age  Tot_Reimbursed_Amt  Tot_Deductible_Amt  DOB_year  DOB_month  \
0         0   66               36060                3264      1943          1   
1         0   65                1250                1250      1944          9   
2         0   74                2390                2390      1935         10   
3         0   77                 870                 870      1932          6   
4         0   77               24450                2586      1932          5   

   DOB_day    ClaimID  Provider  InscClaimAmtReimbursed  AttendingPhysician  \
0        1   CLM46614  PRV55912                   26000               53276   
1        1  CLM724430  PRV55912                     600               73124   
2        1  CLM567180  PRV55912                      20               47058   
3        1  CLM259984  PRV55912                      30               40279   
4        1  CLM565430  PRV55912                      50               36530   

   OperatingPhysician  OtherPhysician  ClmAdmitDiagnosisCode  \
0                   0               0                   3022   
1                   0               0                   3763   
2                   0               0                   3763   
3                   0           31218                   3763   
4                4694               0                   3763   

   DeductibleAmtPaid  DiagnosisGroupCode  ClmDiag

In [5]:
test_df.head(5)

BeneID  Gender  Race  RenalDiseaseIndicator  State  County  \
0  BENE11001       0     0                      0     39     230   
1  BENE11001       0     0                      0     39     230   
2  BENE11007       0     1                      0     45     610   
3  BENE11007       0     1                      0     45     610   
4  BENE11007       0     1                      0     45     610   

   NoOfMonths_PartACov  NoOfMonths_PartBCov  ChronicCond_Alzheimer  \
0                   12                   12                      0   
1                   12                   12                      0   
2                   12                   12                      0   
3                   12                   12                      0   
4                   12                   12                      0   

   ChronicCond_Heartfailure  ChronicCond_KidneyDisease  ChronicCond_Cancer  \
0                         1                          0                   1   
1                         1                          0                   1   
2                         0                          1                   1   
3                         0                          1                   1   
4                         0                          1                   1   

   ChronicCond_ObstrPulmonary  ChronicCond_Depression  ChronicCond_Diabetes  \
0                           1                       0                     0   
1                           1                       0                     0   
2                           1                       1                     0   
3                           1                       1                     0   
4                           1                       1                     0   

   ChronicCond_IschemicHeart  ChronicCond_Osteoporasis  \
0                          0                         1   
1                          0                         1   
2                          1                         0   
3                          1                         0   
4                          1                         0   

   ChronicCond_rheumatoidarthritis  ChronicCond_stroke  OPAnnualDeductibleAmt  \
0                                0                   0                     70   
1                                0                   0                     70   
2                                0                   1                    160   
3                                0                   1                    160   
4                                0                   1                    160   

   Deceased  Age  Tot_Reimbursed_Amt  Tot_Deductible_Amt  DOB_year  DOB_month  \
0         0   66               36060                3264      1943          1   
1         0   66               36060                3264      1943          1   
2         1   69                1490                1490      1940          9   
3         1   69                1490                1490      1940          9   
4         1   69                1490                1490      1940          9   

   DOB_day    ClaimID  Provider  InscClaimAmtReimbursed  AttendingPhysician  \
0        1  CLM392397  PRV55962                      30                6381   
1        1  CLM430760  PRV56112                      30               12407   
2        1  CLM233081  PRV56979                     200               20041   
3        1  CLM496381  PRV56573                      10               14401   
4        1  CLM521391  PRV56573                      10               18681   

   OperatingPhysician  OtherPhysician  ClmAdmitDiagnosisCode  \
0                   0            3608                   2418   
1                   0            6949                   2418   
2                   0           11230                   2418   
3                2703               0                   1408   
4                   0            6974                   2418   

   DeductibleAmtPaid  DiagnosisGroupCode  ClmDiag

In [6]:
fraud_labels.head(5)

Provider PotentialFraud
0  PRV51001             No
1  PRV51003            Yes
2  PRV51004             No
3  PRV51005            Yes
4  PRV51007             No

## # Additional EDA:

- Looking at percentages of fraud vs non-fraud providers in the training dataset

In [7]:
#  Creating df to show percentages of fraud vs non-fraud providers in the dataset:
fraud_df = pd.DataFrame(train_df['PotentialFraud'].value_counts(normalize=True)*100).reset_index()
fraud_df.columns=['Fraud_Labels', 'Per_data']

# Graphically presenting the results
plt.figure(figsize = (12,8))
splot = sns.barplot(x="Fraud_Labels", y="Per_data", data=fraud_df, palette='husl')
for p in splot.patches:
    splot.annotate(format(str(p.get_height().round(2)) + '%'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   size=15,
                   xytext = (0, -20), 
                   textcoords = 'offset points')
plt.xlabel("Fraud Labels", fontsize = 15)
plt.ylabel("Percentage (%)", fontsize = 15)
plt.title("Fraud vs. Non-Fraud Providers - Train data", fontsize = 20)
plt.show()

## Beneficiary focused EDA:

### Looking at chronic conditions for inpatient and outpatient beneficiary claims:

In [8]:
# Creating inpatient subset

inp_df = train_df.loc[train_df.Hospital_Stay >= 0,:]

# Counting all chronic conditions

inp_df['Chr_Cond_Count'] = inp_df['RenalDiseaseIndicator'] + inp_df['ChronicCond_Heartfailure'] + inp_df['ChronicCond_KidneyDisease'] +\
                           inp_df['ChronicCond_Cancer'] + inp_df['ChronicCond_ObstrPulmonary'] + inp_df['ChronicCond_Alzheimer'] +\
                           inp_df['ChronicCond_Depression'] + inp_df['ChronicCond_Diabetes'] + inp_df['ChronicCond_IschemicHeart'] +\
                           inp_df['ChronicCond_Osteoporasis'] + inp_df['ChronicCond_rheumatoidarthritis'] + inp_df['ChronicCond_stroke']

# Looking at mean and median

print(inp_df[['Chr_Cond_Count']].mean())
print(inp_df[['Chr_Cond_Count']].median())

Chr_Cond_Count    5.719771
dtype: float64
Chr_Cond_Count    6.0
dtype: float64


In [9]:
# Creating outpatient subset

out_df = train_df.loc[train_df.Hospital_Stay == -1,:]

# Counting all chronic conditions

out_df['Chr_Cond_Count'] = out_df['RenalDiseaseIndicator'] + out_df['ChronicCond_Heartfailure'] + out_df['ChronicCond_KidneyDisease'] +\
                           out_df['ChronicCond_Cancer'] + out_df['ChronicCond_ObstrPulmonary'] + out_df['ChronicCond_Alzheimer'] +\
                           out_df['ChronicCond_Depression'] + out_df['ChronicCond_Diabetes'] + out_df['ChronicCond_IschemicHeart'] +\
                           out_df['ChronicCond_Osteoporasis'] + out_df['ChronicCond_rheumatoidarthritis'] + out_df['ChronicCond_stroke']

# Looking at mean and median

print(out_df[['Chr_Cond_Count']].mean())
print(out_df[['Chr_Cond_Count']].median())

Chr_Cond_Count    6.774656
dtype: float64
Chr_Cond_Count    7.0
dtype: float64


- Looks like outpatient beneficiaries have slightly higher condition counts

### Looking at visit counts per beneficiary in inpatient/outpatient datasets

In [10]:
inp_df[['BeneID','ClaimID','Chr_Cond_Count']].groupby('BeneID')

BeneID  Gender  Race  RenalDiseaseIndicator  State  County  \
0        BENE11001       0     0                      0     39     230   
5        BENE17521       1     0                      0     39     230   
8        BENE21718       0     0                      0     39     600   
9        BENE22934       1     0                      0     39     280   
10       BENE24402       1     1                      0     39     510   
...            ...     ...   ...                    ...    ...     ...   
558191  BENE149280       1     0                      1     23     240   
558196  BENE151892       0     0                      0      2      20   
558203  BENE154375       0     0                      1      7      30   
558205  BENE156181       0     0                      1     37     260   
558210  BENE159166       1     0                      0      5     480   

        NoOfMonths_PartACov  NoOfMonths_PartBCov  ChronicCond_Alzheimer  \
0                        12                   12                      0   
5                        12                   12                      0   
8                        12                   12                      1   
9                        12                   12                      1   
10                       12                   12                      0   
...                     ...                  ...                    ...   
558191                   12                   12                      0   
558196                   12                   12                      1   
558203                   12                   12                      1   
558205                   12                   12                      0   
558210                   12                   12                      1   

        ChronicCond_Heartfailure  ChronicCond_KidneyDisease  \
0                              1                          0   
5                              0                          0   
8                              1                          1   
9                              0                          1   
10                             1                          1   
...                          ...                        ...   
558191                         0                          0   
558196                         1                          1   
558203                         0                          0   
558205                         0                          0   
558210                         0                          1   

        ChronicCond_Cancer  ChronicCond_ObstrPulmonary  \
0                        1                           1   
5                        1                           0   
8                        1                           1   
9                        1                           0   
10                       0                           1   
...                    ...                         ...   
558191                   0                           0   
558196                   1                           1   
558203                   1                           1   
558205                   0                           1   
558210                   0                           1   

        ChronicCond_Depression  ChronicCond_Diabetes  \
0                            0                     0   
5                            1                     1   
8                            1                     1   
9                            1                     0   
10                           1                     1   
...                        ...                   ...   
558191                       0                     0   
558196                       1                     0   
558203                       1                     0   
558205                       1                     0   
558210                       0                     0   

        ChronicCond_IschemicHeart  ChronicCond_Osteoporasis  \
0                               

In [11]:
out_df

BeneID  Gender  Race  RenalDiseaseIndicator  State  County  \
1        BENE12511       0     0                      0     39     510   
2        BENE14577       1     0                      1     33      70   
3        BENE15074       1     0                      0     39     470   
4        BENE16973       0     0                      0     39     310   
6        BENE17795       1     0                      0     39     320   
...            ...     ...   ...                    ...    ...     ...   
558204  BENE154687       0     1                      0     25     150   
558206  BENE157252       0     0                      0     21     230   
558207  BENE157378       1     0                      1      5     200   
558208  BENE158295       1     0                      0     16     490   
558209  BENE158736       1     0                      0     43     170   

        NoOfMonths_PartACov  NoOfMonths_PartBCov  ChronicCond_Alzheimer  \
1                        12                   12                      1   
2                        12                   12                      1   
3                        12                   12                      0   
4                        12                   12                      0   
6                        12                   12                      1   
...                     ...                  ...                    ...   
558204                   12                   12                      0   
558206                   12                   12                      1   
558207                   12                   12                      0   
558208                   12                   12                      1   
558209                   12                   12                      1   

        ChronicCond_Heartfailure  ChronicCond_KidneyDisease  \
1                              1                          1   
2                              0                          0   
3                              1                          0   
4                              0                          0   
6                              0                          1   
...                          ...                        ...   
558204                         1                          0   
558206                         1                          0   
558207                         0                          0   
558208                         0                          0   
558209                         1                          1   

        ChronicCond_Cancer  ChronicCond_ObstrPulmonary  \
1                        1                           1   
2                        1                           1   
3                        1                           1   
4                        1                           1   
6                        1                           0   
...                    ...                         ...   
558204                   1                           1   
558206                   0                           0   
558207                   1                           0   
558208                   0                           0   
558209                   0                           1   

        ChronicCond_Depression  ChronicCond_Diabetes  \
1                            1                     0   
2                            1                     1   
3                            1                     0   
4                            1                     1   
6                            0                     0   
...                        ...                   ...   
558204                       1                     0   
558206                       1                     0   
558207                       1                     0   
558208                       0                     0   
558209                       1                     0   

        ChronicCond_IschemicHeart  ChronicCond_Osteoporasis  \
1                               